In [57]:
import xmltodict
import pandas as pd
from tqdm.auto import tqdm

In [2]:
b = open('data/drought.xml', "r").readlines()

In [20]:
latlon_df = pd.read_html('https://en.wikipedia.org/wiki/User:Michael_J/County_table', index_col=0)[0]
float_degrees = lambda x: float(x.replace('°','').replace('–','-'))
latlon_df['Latitude'] = latlon_df['Latitude'].apply(float_degrees)
latlon_df['Longitude'] = latlon_df['Longitude'].apply(float_degrees)

In [46]:
def df_from_line(line):
    values = (
        xmltodict.parse(line.replace('@','_'))
        ['item']
        ['ArrayOfCountyStatisticsDroughtSeverity']
        ['CountyStatisticsDroughtSeverity']
        ['value']
    )
    fips = int(values[0]['FIPS'])
    fips_county = latlon_df[latlon_df['FIPS']==fips]
    lat, lon = fips_county['Latitude'].values[0], fips_county['Longitude'].values[0]
    val_dict = {
        'fips': [],
        'lat': [],
        'lon': [],
        'None': [],
        'D0': [],
        'D1': [],
        'D2': [],
        'D3': [],
        'D4': [],
        'date': [],
    }
    for val in values:
        val_dict['fips'].append(fips)
        val_dict['lat'].append(lat)
        val_dict['lon'].append(lon)
        val_dict['None'].append(val['None'])
        val_dict['D0'].append(val['D0'])
        val_dict['D1'].append(val['D1'])
        val_dict['D2'].append(val['D2'])
        val_dict['D3'].append(val['D3'])
        val_dict['D4'].append(val['D4'])
        val_dict['date'].append(val['ValidStart'])
    df = pd.DataFrame.from_dict(val_dict)
    df['None'] = df['None'].astype(float)
    df['D0'] = df['D0'].astype(float)
    df['D1'] = df['D1'].astype(float)
    df['D2'] = df['D2'].astype(float)
    df['D3'] = df['D3'].astype(float)
    df['D4'] = df['D4'].astype(float)
    df['date'] = pd.to_datetime(df['date'])
    return df

In [47]:
df_from_line(b[6]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fips    1096 non-null   int64         
 1   lat     1096 non-null   float64       
 2   lon     1096 non-null   float64       
 3   None    1096 non-null   float64       
 4   D0      1096 non-null   float64       
 5   D1      1096 non-null   float64       
 6   D2      1096 non-null   float64       
 7   D3      1096 non-null   float64       
 8   D4      1096 non-null   float64       
 9   date    1096 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 85.8 KB


In [58]:
dfs = []
for line in tqdm(b[2:-1]):
    dfs.append(df_from_line(line))

  0%|          | 0/3109 [00:00<?, ?it/s]

In [60]:
all_df = pd.concat(dfs)

In [61]:
all_df.to_pickle('drought_df.pkl')